In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
model='/content/drive/MyDrive/gemma'

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 35.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [4]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [5]:
import pandas as pd
from datasets import load_metric
from transformers import AutoTokenizer
from unsloth import FastLanguageModel


In [6]:
max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

In [7]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

Unsloth 2024.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [8]:
# Load the dataset
test_df = pd.read_csv('NLP_Recipe_test.csv')  # Ensure this path is correct

# Load the BLEU metric
bleu_metric = load_metric('bleu')

# Function to generate recipes and compute BLEU scores
def generate_and_score(index, row):
    ner_input = row['ner']  # Key ingredients input
    true_output = row['ingredients'] + " " + row['steps']  # True combined output

    # Prepare the prompt
    recipe_prompt = f"""Given the following key ingredients, generate the full ingredient list with quantities and cooking steps:

    ### Key Ingredients:
    {ner_input}

    ### Full Ingredients and Steps:
    """

    # Tokenize and generate the output
    inputs = tokenizer(
        [recipe_prompt],
        return_tensors="pt"
    ).to('cuda')

    outputs = model.generate(**inputs, max_new_tokens=512, use_cache=True)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the generated recipe part after "### Full Ingredients and Steps:"
    generated_recipe = generated_text.split("### Full Ingredients and Steps:\n")[1].strip()

    # Compute BLEU score
    reference = [true_output.split()]
    candidate = generated_recipe.split()
    bleu_score = bleu_metric.compute(predictions=[candidate], references=[reference])

    return bleu_score['bleu'], generated_recipe

# Iterate over the first 5 rows and compute BLEU scores
for index, row in test_df.head(5).iterrows():
    score, recipe = generate_and_score(index, row)
    print(f"Row {index} BLEU Score: {score}")
    print("Generated Recipe and Steps:", recipe)
    print("----------------------------------------")

<ipython-input-8-1d6f817b981b>:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric('bleu')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.0/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Row 0 BLEU Score: 0.27116284593923873
Generated Recipe and Steps: 100.0 gram spaghetti, 100.0 gram kipfilet, 100.0 gram kerstomaten, 1.0 tablespoon basilicum, 1.0 tablespoon margarine, 1.0 bag knorr kruidenpasta spaghetti bolognese kook de spaghetti volgens de aanwijzingen op de verpakking . snijd de kipfilet in blokjes . snijd de kerstomaten in plakjes . verwarm de margarine in een pan en bak hierin de kipfilet goudbruin . voeg de knorr kruidenpasta spaghetti bolognese toe en bak deze even mee . voeg de kerstomaten toe en laat alles even op laag vuur doorkoken . schep de pasta door de saus en serveer direct.
----------------------------------------
Row 1 BLEU Score: 0.10524408019697566
Generated Recipe and Steps: 1 whole chicken without giblets, 1 small handful of fresh rosemary sprigs, 1 small handful of fresh thyme sprigs, peel from 1 small lemon, sliced, 2 garlic cloves, 2 large onions, peeled and sliced, salt and pepper, 1/4 cup olive oil, 1/4 cup honey, divided, 1/2 cup white win